In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from skimage.transform import estimate_transform

Setting environment for Gordon


In [2]:
from collections import defaultdict

In [3]:
def convert_polygon_v1_to_v2(polygon):

    new_polygon = {}
    label_split = polygon['label'].split('_')
    new_polygon['label'] = label_split[0]
    if len(label_split) == 2:
        if label_split[1] == 'L':
            new_polygon['side'] = 'L'
        else:
            new_polygon['side'] = 'R'
    else:
        new_polygon['side'] = None
    
    new_polygon['vertices'] = polygon['vertices']
    new_polygon['labelPos'] = polygon['labelPos']
#     new_polygon['section'] = sec
    
    return new_polygon

In [51]:
# for stack in ['MD589', 'MD594', 'MD585']:

stack = 'MD594'
# stack = 'MD589'

labelings_v2_dir = create_if_not_exists('/oasis/projects/nsf/csd395/yuncong/CSHL_labelings_v2/' + stack)

timestamp = datetime.datetime.now().strftime("%m%d%Y%H%M%S")
username = 'yuncong'

new_labeling = dict(indexing_scheme = 'section',
                    downsample = 1,
                    orientation = 'sagittal',
                    username = username,
                    timestamp = timestamp,
                    polygons = defaultdict(list))

for sec in range(section_number_lookup[stack]):

    try:
        polygons = DataManager.load_annotation(stack=stack, section=sec, username=username, 
            annotation_rootdir='/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/')[0]
    except:
        sys.stderr.write('Cannot read labeling for section %d \n.' % sec)
        continue

#     new_polygons = [convert_polygon_v1_to_v2(polygon) for polygon in polygons]
    new_polygons = map(convert_polygon_v1_to_v2, polygons)    
    new_labeling['polygons'][sec] = new_polygons

Directory /oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/MD594/0000/ does not exist.
Cannot read labeling for section 0 
.Directory /oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/MD594/0001/ does not exist.
Cannot read labeling for section 1 
.Directory /oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/MD594/0002/ does not exist.
Cannot read labeling for section 2 
.Directory /oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/MD594/0003/ does not exist.
Cannot read labeling for section 3 
.Directory /oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/MD594/0004/ does not exist.
Cannot read labeling for section 4 
.Directory /oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/MD594/0005/ does not exist.
Cannot read labeling for sect

In [52]:
labeling, _, _ = DataManager.load_annotation_v2(stack=stack, username='Lauren', orientation=new_labeling['orientation'],
                                          annotation_rootdir='/home/yuncong/csd395/CSHL_labelings_new/')

In [53]:
for i, polygons in labeling.iteritems():
    for p in polygons:
        sec = p['section']
        p2 = {'label': p['label'], 'vertices': [(int(x), int(y)) for x,y in p['vertices']], 'side': None}
        if sec in new_labeling['polygons']:
            matched_polygons = [pp for pp in new_labeling['polygons'][sec] if pp['label'] == p['label']]
            
            for matched_polygon in matched_polygons:
                new_labeling['polygons'][sec].remove(matched_polygon)                
        p2['labelPos'] = np.mean(p2['vertices'], axis=0).astype(np.int)
        new_labeling['polygons'][sec].append(p2)

In [54]:
for p in new_labeling['polygons'][166]:
    print p['label']

7n
LPB
Pn
5N
VLL
APT
LRt
7n
LSO
SNCD
MVeMC
LVPO
LVe
mcp
PPT
ECu
sp5
MiTg
tz
DPO
SuVe
MPB
SpVe
Sol
PTg
7n
7n
7N
CnF
SNCD
scp
7n
SNR
SNC
IC
SC


In [55]:
new_fn = '%(stack)s_%(orient)s_downsample%(downsample)d_%(username)s_%(timestamp)s.pkl' % \
            dict(stack=stack, orient=new_labeling['orientation'], downsample=new_labeling['downsample'], 
                 username='yuncong', timestamp=timestamp)

pickle.dump(new_labeling, open(os.path.join(labelings_v2_dir, new_fn), 'w'))

In [ ]:
! mkdir /home/yuncong/csd395/CSHL_labelings_new/MD589
! scp 132.239.73.151:CSHL_labelings_new/MD589/MD589_sagittal_downsample1_Lauren_08162016173458.pkl \
    /home/yuncong/csd395/CSHL_labelings_new/MD589

In [ ]:
! mkdir /home/yuncong/csd395/CSHL_labelings_new/MD594
! scp 132.239.73.151:CSHL_labelings_new/MD594/MD594_sagittal_downsample1_Lauren_08162016213139.pkl \
    /home/yuncong/csd395/CSHL_labelings_new/MD594